In [1]:
# basic
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import time
import glob
import os
from numpy.linalg import inv
from bidict import bidict

# user-defined
import sys
sys.path.append('/Users/cheng/Google Drive/PhD/Research/Portfolio Selection via TBN/codes/')
from module import *
from module.backtesting import *
from module.agent_network import *
from module.environment import market_envrionment
from module.data_handler import *
import tensorflow as tf
tf.config.run_functions_eagerly(True)

# visulization
import igraph
import cairocffi
# import cairo
import matplotlib.pyplot as plt

# 
from scipy.sparse.csgraph import minimum_spanning_tree

In [4]:
bt = vectorized_backtesting()
db = data_handler()
np.random.seed(0)

In [5]:
DIJA_subset_gvkey_file_path = '/Users/cheng/Google Drive/PhD/Research/Portfolio Selection via TBN/data/Data/gvkey_ticker.csv'
DIJA_subset_gvkey_df = pd.read_csv(DIJA_subset_gvkey_file_path, header=0, index_col=0, engine='c')

In [8]:
DIJA_subset_gvkey_mapping = { gvkey:ticker for gvkey, ticker in zip(DIJA_subset_gvkey_df.gvkey, DIJA_subset_gvkey_df.TICKER)}

In [9]:
crsp_subset_gvkey = dh.company_gvkey_vector
DIJA_subset_gvkey = DIJA_subset_gvkey_df.gvkey

In [11]:
DIJA_missing_gvkey = [gvkey for gvkey in DIJA_subset_gvkey if gvkey not in crsp_subset_gvkey]

In [15]:
DIJA_missing_ticker = [DIJA_subset_gvkey_mapping[gvkey] for gvkey in DIJA_missing_gvkey]

In [17]:
set(DIJA_missing_ticker)

{'CRM',
 'CSCO',
 'CVX',
 'DOW',
 'GS',
 'HD',
 'HON',
 'JPM',
 'NKE',
 'TRV',
 'UNH',
 'V',
 'VZ',
 'WBA',
 'WMT'}

In [3]:
sample_size = 100
company_subset_gvkey_vector_100 = np.random.choice(db.company_gvkey_vector, sample_size, replace=False)
db.extract_TNIC_network_subset(key_list= company_subset_gvkey_vector_100,
                                file_input_path='/Users/cheng/Documents/Research Data/Text Base Network/new/tnic_all_data/',
                                file_output_path='/Users/cheng/Google Drive/PhD/Research/Portfolio Selection via TBN/data/TBN/sample_company_100/')

100%|██████████| 32/32 [16:31<00:00, 30.98s/it]


In [4]:
sample_size = 50
company_subset_gvkey_vector_50 = np.random.choice(db.company_gvkey_vector, sample_size, replace=False)
db.extract_TNIC_network_subset(key_list= company_subset_gvkey_vector_50,
                                file_input_path='/Users/cheng/Documents/Research Data/Text Base Network/new/tnic_all_data/',
                                file_output_path='/Users/cheng/Google Drive/PhD/Research/Portfolio Selection via TBN/data/TBN/sample_company_50/')

100%|██████████| 32/32 [09:58<00:00, 18.70s/it]


In [6]:
sample_size = 5
company_subset_gvkey_vector_5 = np.random.choice(db.company_gvkey_vector, sample_size, replace=False)
db.extract_TNIC_network_subset(key_list= company_subset_gvkey_vector_5,
                                file_input_path='/Users/cheng/Documents/Research Data/Text Base Network/new/tnic_all_data/',
                                file_output_path='/Users/cheng/Google Drive/PhD/Research/Portfolio Selection via TBN/data/TBN/sample_company_5/')

100%|██████████| 32/32 [04:59<00:00,  9.37s/it]


In [32]:
stock_return_supp_file_path = '/Users/cheng/Google Drive/PhD/Research/Portfolio Selection via TBN/data/Data/permno_ret2.csv'
stock_return_supp = pd.read_csv(stock_return_supp_file_path, index_col=[0,1])

In [38]:
stock_return_supp = stock_return_supp.unstack().T

In [42]:
stock_date_format = '%m/%d/%Y' 
stock_return_supp.index = [datetime.strptime(x, stock_date_format) for _, x in stock_return_supp.index]

In [48]:
stock_return_supp = stock_return_supp.sort_index()

In [61]:
stock_return_original_file_path = '/Users/cheng/Google Drive/PhD/Research/Portfolio Selection via TBN/data/Data/permno_ret.csv'
stock_return_origin = pd.read_csv(stock_return_original_file_path, header=0, index_col=0)

In [68]:
stock_date_format = '%m/%d/%y' 
stock_return_origin = stock_return_origin.dropna(axis='columns') # drop incomplete data to make TBN consitency cross year
stock_return_origin.columns = [int(x) for x in stock_return_origin.columns]
stock_return_origin.index = [datetime.strptime(x, stock_date_format) for x in stock_return_origin.index]

In [72]:
result = pd.concat([stock_return_origin, stock_return_supp], axis=1)

In [74]:
output_path = '/Users/cheng/Google Drive/PhD/Research/Portfolio Selection via TBN/data/Data/stock_return_crsp.csv'
result.to_csv(output_path)

In [2]:
TBN_DIJA_file_path = '/Users/cheng/Google Drive/PhD/Research/Portfolio Selection via TBN/data/TBN/sample_company_30/TBN_2010.csv'
TBN_DIJA = pd.read_csv(TBN_DIJA_file_path, header=0, index_col=0)
DIJA_subset_gvkey_list = [int(gvkey) for gvkey in TBN_DIJA.columns]

In [3]:
dh = data_handler()
sample_gvkey_list = dh.PERMNO_to_gvkey_mapping.values()


/Users/cheng/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (147,155,156,157,158) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
59459 in dh.company_PERMNO_vector

True

In [13]:
dh.PERMNO_to_gvkey_mapping[70519]

3243

In [12]:
[gvkey for gvkey in DIJA_subset_gvkey_list if gvkey not in dh.company_gvkey_vector]

[3243]

In [ ]:
DIJA_subset_gvkey_list

In [4]:
dh.extract_TNIC_network_subset(key_list=DIJA_subset_gvkey_list, file_output_path='/Users/cheng/Google Drive/PhD/Research/Portfolio Selection via TBN/data/TBN/sample_djia_23/')

100%|██████████| 32/32 [06:54<00:00, 12.96s/it]
